In [46]:
import uproot as up
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-v0_8-paper')
plt.rcParams["patch.force_edgecolor"] = True


In [47]:
file_BKG = up.open("radioactivity_userfile_7days.root")
#file_IBD = up.open('../BrutalCuts/unoscillated_IBD_userfile.root') 
file_IBD = up.open('ibd_userfile_7days.root') 


dataset_IBD_all = file_IBD['TRec'].arrays(library = 'np')
dataset_BKG = file_BKG['TRec'].arrays(library = 'np')

dataset_IBD = {}
for key in ['recx', 'recy', 'recz', 'm_QEn', 'm_triggerT']:
    dataset_IBD[key] = dataset_IBD_all[key]

en_fact = 0.92
dataset_IBD["m_QEn"] = dataset_IBD["m_QEn"]*en_fact 

In [48]:
dataset_IBD

{'recx': array([-6546.7075, -5635.7627, 13160.146 , ..., -5190.015 , 12472.224 ,
        12667.809 ], dtype=float32),
 'recy': array([12444.845 , 11995.087 ,  5150.4067, ...,  3430.7043, -9249.245 ,
        -9773.4   ], dtype=float32),
 'recz': array([-10109.446 , -10101.866 ,   7890.3755, ..., -14862.485 ,
         -4518.004 ,  -4311.2036], dtype=float32),
 'm_QEn': array([2.2872338, 2.1975338, 5.281242 , ..., 2.2772105, 3.309503 ,
        2.1785235], dtype=float32),
 'm_triggerT': array([7.60000000e+01, 2.52610000e+05, 1.20811085e+09, ...,
        2.03446236e+12, 2.03455463e+12, 2.03455477e+12])}

In [49]:
xs_reco = file_BKG['TRec']['recx'].array(library = 'np')
ys_reco = file_BKG['TRec']['recy'].array(library = 'np')
zs_reco = file_BKG['TRec']['recz'].array(library = 'np')
es_reco_BKG = file_BKG['TRec']['m_QEn'].array(library = 'np')

rs_reco = np.sqrt(xs_reco**2 + ys_reco**2 + zs_reco**2)

# Cuts

In [60]:
from numba import njit, prange #prange serve per fare un ciclo for ma eseguito come in c++, molto più veloce


@njit(parallel = True) #il parallel serve per far lavorararre tutti i core in parallelo
def selection(x,y,z,E,t,delta_time = 1e6, delta_radius = 1500, min_energy_prompt = 0.7,max_energy_prompt = 12, min_energy_delay = 1.9, max_energy_delay = 2.5, min_energy_delay_carb = 4.4, max_energy_delay_carb = 5.5):
    prompt_columns = np.zeros(x.shape)
    delay_columns = np.zeros(x.shape)
    delay_columns_carb = np.zeros(x.shape)

    for i_pro in prange(x.shape[0]-1):
        i_del = i_pro + 1


        if t[i_del] - t[i_pro] < delta_time and (t[i_del] - t[i_pro]) > 0: 
            if np.sqrt((x[i_del] - x[i_pro])**2 + (y[i_del] - y[i_pro])**2 + (z[i_del] - z[i_pro])**2) < delta_radius: #radius cut
                if E[i_pro]>= min_energy_prompt and E[i_pro]<= max_energy_prompt:
                    if E[i_del]>= min_energy_delay and E[i_del]<= max_energy_delay:
                        prompt_columns[i_pro] = 1
                        delay_columns[i_del] = 1
                    if E[i_del]>= min_energy_delay_carb and E[i_del]<= max_energy_delay_carb:
                        prompt_columns[i_pro] = 1
                        delay_columns_carb[i_del] = 1

    return prompt_columns,delay_columns,delay_columns_carb
    

    


In [61]:
cut = 100000
prompt_columns, delay_columns, delay_columns_carb = selection(dataset_IBD['recx'][:cut],
                                                              dataset_IBD['recy'][:cut],
                                                              dataset_IBD['recz'][:cut],
                                                              dataset_IBD["m_QEn"][:cut],
                                                              dataset_IBD["m_triggerT"][:cut])

In [62]:
print(prompt_columns.sum()*2)
print(prompt_columns.shape[0])



96240.0
100000


In [58]:
print("Accuracy: ", ( (prompt_columns.sum() + delay_columns.sum()+ delay_columns_carb.sum())  / prompt_columns.shape[0] ) * 100)

Accuracy:  80.738


ACTUNG: Qui non fai la tabella delle features, ma fai solo una differenza tra il un evento e SOLAMENTE quello di dopo.